In [0]:
"""
We use following lines because we are running on Google Colab
If you are running notebook on a local computer, you don't need this cell
"""
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.chdir('/content/gdrive/My Drive/finch/tensorflow1/free_chat/chinese/main')

In [0]:
from collections import Counter
import random

In [0]:
params = {
    'min_freq': 5,
    'test_size': 0.01,
    'vocab_path': '../vocab/char.txt',
    'train_path': '../data/train.txt',
    'test_path': '../data/test.txt',
}

In [0]:
char_counter = Counter()

with open('../data/xiaohuangji50w_nofenci.conv') as f:
  single_turns = f.read().split('E')

with open(params['train_path'], 'w') as f_train, open(params['test_path'], 'w') as f_test:
  for t in single_turns:
    li = t.strip().split('M ')
    if len(li) == 3:
      q, a = li[1:]
      q = q.strip()
      a = a.strip()
      
      if random.random() < params['test_size']:
        f_test.write(q+'|'+a+'\n')
      else:
        f_train.write(q+'|'+a+'\n')
      char_counter.update(list(q))
      char_counter.update(list(a))

chars = ['<pad>', '<start>', '<end>'] + [char for char, freq in char_counter.most_common() if freq >= params['min_freq']]
with open(params['vocab_path'], 'w') as f:
  for c in chars:
    f.write(c+'\n')